In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os



In [ ]:
learning_rate = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
data = []
labels = []

In [ ]:

# Collection of data and its labels...

Image_Paths_1 = list(paths.list_images(r"C:\Users\sudha\Desktop\data\with_mask"))
Image_Paths_2 = list(paths.list_images(r"C:\Users\sudha\Desktop\data\without_mask"))

for i in Image_Paths_1:
    labels.append("with_mask")
for i in Image_Paths_2:
    labels.append("without_mask")

Image_Paths = Image_Paths_1 + Image_Paths_2

for image_path in Image_Paths:
    image = load_img(image_path,target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

data = np.array(data, dtype="float32")
labels = np.array(labels)


In [ ]:
#Encoding of labels using OneHotEncoding

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

print(labels)

print()

print(data)

In [ ]:
#For data Augumentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# train and test data split
train_X,test_X,train_Y,test_Y = train_test_split(data,labels,test_size=0.2,stratify=labels,random_state=42)

In [ ]:
#Contruction of model
# Here we use MobilNetV2 model
# We contruct the Fully Connected head by ourselves
# and append it in the model

baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# We use adam optimzer
# Then we start compiling our model

print("compiling model...")

opt = Adam(lr=learning_rate, decay=learning_rate / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])


In [ ]:
print("training head...")
H = model.fit(
	aug.flow(train_X, train_Y, batch_size=BS),
	steps_per_epoch=len(train_X) // BS,
	validation_data=(test_X, test_Y),
	validation_steps=len(test_X) // BS,
	epochs=EPOCHS)

In [ ]:
#Graphical representation of our model parameters

N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()


In [ ]:
# After training the model we start testing it using webcam 
# We using face detection algorithm and then predict the presence of mask using our model
# If rectangle color = red, means no mask
# If rectangle color = green, means there is mask
# The percentage shows the confidence of the model in its prediction

cap = cv.VideoCapture(0)
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
while True:
    ret,frame = cap.read()
    facee = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    #print(facee.shape)
    facee = cv.resize(facee, (224, 224))
    #print(facee.shape)
    facee = img_to_array(facee)
    #print(facee)
    facee = preprocess_input(facee)
    #print(facee)
    facee = np.array(facee, dtype="float32")
    #print(facee.shape)
    facee = facee.reshape(1,224,224,3)
    preds = model.predict(facee,batch_size=BS)
    [mask, withoutMask] = preds[0]
    gry = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gry, 1.2, 5)
    color = (0,0,255)
    for(x,y,w,h) in faces:
        l = "mask" if mask > withoutMask else "No mask"
        color = (0, 255, 0) if l == "mask" else (0, 0, 255)
        maxim = "{}: {:.2f}%".format(l, max(mask, withoutMask) * 100)
        cv.putText(frame, maxim, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        cv.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    cv.imshow("frame",frame)

    if cv.waitKey(1000) == 27:
        break
cap.release()
cv.destroyAllWindows()